# Day 06

## Read input

In [3]:
with open("input.txt") as fh:
    groups = []
    
    group_answers = {"count": 0, "answers": {}}
    for row in fh.readlines():
        row = row.strip()
        if row == "":
            groups.append(group_answers)
            group_answers = {"count": 0, "answers": {}}
            continue
        for c in row:
            try:
                group_answers["answers"][c] += 1
            except KeyError:
                group_answers["answers"][c] = 1
        group_answers["count"] += 1
    groups.append(group_answers)

## Part 1

In [4]:
sum([len(set(group["answers"].keys())) for group in groups])

6778

## Part 2

In [5]:
group_answers

{'count': 3,
 'answers': {'u': 3, 'k': 3, 'r': 3, 'p': 3, 'f': 3, 'l': 3, 'a': 3}}

In [12]:
sum([item for sublist in [[group["answers"][answer] == group["count"] for answer in group["answers"]] for group in groups] for item in sublist])

3406

## Trying GPU

In [13]:
import string

In [17]:
with open("input.txt") as fh:
    groups = []
    
    group_answers = {letter: 0 for letter in list(string.ascii_lowercase)}
    group_answers["count"] = 0
    
    for row in fh.readlines():
        row = row.strip()
        if row == "":
            groups.append(group_answers)
            group_answers = {letter: 0 for letter in list(string.ascii_lowercase)}
            group_answers["count"] = 0
            continue
            
        for c in row:
            group_answers[c] += 1
        group_answers["count"] += 1
    groups.append(group_answers)

In [20]:
import pandas as pd
import cudf

In [21]:
df = cudf.from_pandas(pd.DataFrame(groups))
df

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,count
0,1,0,1,4,0,4,0,1,4,0,...,4,0,4,4,4,4,4,0,0,4
1,1,2,0,0,0,0,0,0,0,1,...,0,2,1,0,2,0,0,0,1,2
2,0,0,0,2,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,5
3,0,0,0,1,3,1,0,0,0,3,...,3,0,0,0,1,2,3,0,0,3
4,2,2,2,2,2,1,1,1,2,1,...,1,1,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,4,1,4,1,0,1,1,1,4,1,...,1,0,1,2,1,1,0,0,1,4
484,3,0,3,3,3,3,3,0,0,3,...,3,3,3,3,3,3,0,3,0,3
485,1,1,3,2,1,1,1,1,2,1,...,1,1,1,1,0,1,1,1,1,3
486,1,2,1,0,0,2,1,5,0,1,...,2,0,1,1,1,1,1,1,1,5


In [28]:
groups_array = df.as_gpu_matrix()

## Part 1

In [31]:
from numba import cuda
import numpy as np

In [35]:
@cuda.jit
def count_group_answers(groups_array, output):
    pos = cuda.grid(1)
    if pos < len(groups_array):
        for answer in groups_array[pos][0:-1]:
            if answer > 0:
                output[pos] += 1

In [36]:
threadsperblock = 128
blockspergrid = (len(groups_array) + (threadsperblock - 1)) // threadsperblock

In [37]:
output = np.zeros(len(groups_array))
count_group_answers[blockspergrid, threadsperblock](groups_array, output)
output

array([16., 10.,  3., 12., 25.,  8., 11., 11., 12., 24.,  5., 19., 19.,
       12., 26., 10., 18.,  7., 15.,  9.,  5., 24., 26., 15., 13.,  4.,
       26.,  9., 18., 20., 12., 16., 17., 11., 19., 21., 11., 10., 17.,
        9.,  6., 14.,  7.,  9., 10.,  2., 19., 24., 24., 17.,  9., 13.,
       13., 16.,  1.,  6., 21., 13., 22.,  1., 17., 16., 22., 12.,  5.,
       13.,  9.,  3.,  7.,  8., 21., 24.,  7., 16.,  3., 20., 17.,  3.,
       21., 10., 17.,  9., 14., 26.,  2., 23., 20., 18.,  4.,  5., 26.,
       14., 17., 24., 22., 12., 25.,  2., 20., 10., 15., 23., 13., 23.,
       15.,  7., 22.,  1., 18., 13., 22., 13.,  8.,  3., 25., 22.,  3.,
       13.,  5., 17., 12., 17., 24., 11., 14., 11.,  2.,  7.,  8.,  2.,
       14.,  7., 23., 21.,  7., 14., 10.,  4., 15., 19., 21.,  6., 11.,
       25., 21., 20., 22., 13., 24., 24.,  3., 14., 21.,  9., 25.,  4.,
       26., 23., 14., 13., 11., 25., 21., 26., 24.,  3., 21.,  7., 21.,
       18., 25., 21.,  7.,  3., 17.,  5., 19., 14., 17., 10., 10

In [38]:
sum(output)

6778.0

## Part 2

In [39]:
@cuda.jit
def count_group_all_answers(groups_array, output):
    pos = cuda.grid(1)
    if pos < len(groups_array):
        for answer in groups_array[pos][0:-1]:
            if answer == groups_array[pos][-1]:
                output[pos] += 1

In [40]:
output = np.zeros(len(groups_array))
count_group_all_answers[blockspergrid, threadsperblock](groups_array, output)
output

array([11.,  3.,  0.,  5., 19.,  2.,  0.,  6.,  9., 21.,  0., 14., 18.,
       10., 26.,  3., 13.,  5.,  3.,  8.,  2.,  2., 26.,  3., 10.,  3.,
        2.,  1., 11., 11.,  1.,  5.,  6.,  2.,  9., 19.,  6.,  5., 10.,
        2.,  2.,  1.,  4.,  8.,  0.,  2., 13., 23., 21.,  5.,  3., 12.,
        4., 16.,  1.,  1.,  7.,  8.,  2.,  1.,  9., 12., 17.,  3.,  5.,
        2.,  4.,  2.,  1.,  6.,  1.,  7.,  2., 10.,  0., 12., 14.,  3.,
       11., 10., 11.,  8.,  0., 12.,  2.,  8.,  4., 11.,  4.,  0., 24.,
       14.,  0., 12.,  4.,  7.,  9.,  0.,  8.,  2.,  0., 17., 12.,  8.,
        5.,  3.,  0.,  1.,  6., 13., 10.,  7.,  7.,  1.,  4.,  1.,  0.,
       10.,  4., 14.,  3., 13., 12., 11., 12.,  4.,  0.,  0.,  3.,  0.,
       13.,  5., 11., 21.,  3.,  1.,  9.,  2.,  2.,  2., 10.,  2., 10.,
       10.,  3.,  1., 11.,  9., 21., 11.,  3.,  5., 20.,  0.,  5.,  3.,
        6.,  8.,  5., 13.,  4., 10., 14.,  5., 15.,  2.,  8.,  5.,  1.,
        1., 15.,  5.,  4.,  1., 14.,  3.,  9.,  7.,  3.,  3.,  9

In [41]:
sum(output)

3406.0